# 自製智能中文選字系統  (1)

In [1]:
import sys
sys.version

'3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]'

## 資料前處理

確認版本為 python3

In [2]:
import re

In [3]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    chinese_pattern = r'[\u4E00-\u9FFF]+'
    segments = re.findall(chinese_pattern, line, flags=re.UNICODE)
    # END YOUR CODE
    return segments

In [4]:
segments = []
with open('./wiki_zh_small.txt', encoding = 'utf-8') as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

In [5]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

## Ngram

一開始要先計算字詞出現的次數

In [6]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]  # 分別代表計算0、1、...個字的出現次數

    def fit(self, segments):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數
        # YOUR CODE HERE
        for segment in segments:
            for i in range(1, self.n+1):
                self.counters[i].update(self._skip(segment, i))
        self.counters[0] = Counter({'': sum(self.counters[1].values())})
        # END YOUR CODE

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [7]:
counters = Counters(n=3)
counters.fit(segments)

In [8]:
counters[0]
# 應該為： Counter({'': 371373})

Counter({'': 371373})

In [9]:
counters[2]

Counter({'英語': 175,
         '語英': 1,
         '又稱': 31,
         '稱爲': 223,
         '爲英': 10,
         '英文': 37,
         '是一': 245,
         '一種': 249,
         '種西': 1,
         '西日': 5,
         '日耳': 15,
         '耳曼': 15,
         '曼語': 19,
         '語言': 463,
         '誕生': 19,
         '生於': 23,
         '於中': 74,
         '中世': 20,
         '世紀': 245,
         '紀早': 4,
         '早期': 68,
         '期的': 71,
         '的英': 26,
         '英格': 5,
         '格蘭': 13,
         '如今': 14,
         '今具': 1,
         '具有': 128,
         '有全': 3,
         '全球': 75,
         '球通': 1,
         '通用': 31,
         '用語': 13,
         '語的': 71,
         '的地': 108,
         '地位': 70,
         '一詞': 48,
         '詞源': 11,
         '源於': 49,
         '於遷': 1,
         '遷居': 3,
         '居英': 2,
         '蘭的': 6,
         '的日': 34,
         '曼部': 1,
         '部落': 17,
         '落盎': 1,
         '盎格': 16,
         '格魯': 15,
         '得名': 14,
         '名於': 4,
         '於臨': 1,
         '臨波': 1,
  

In [10]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        # YOUR CODE HERE
        minor_prefix = prefix[len(prefix)-self.n+1::]
        minor_count = self.minor_counter.get(minor_prefix)
        sorted_probs = [(v / minor_count, k[-1]) for k, v in self.major_counter.items() if k[:self.n-1] == minor_prefix]
        sorted_probs = sorted(sorted_probs, reverse = True)
        # END YOUR CODE
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}


In [11]:
unigram = Ngram(1, counters)

In [12]:
unigram.predict_proba('我思')
# 應該為：[(0.035732269174118744, '的'),
#         (0.012927703414087723, '國'),
#         (0.010620050461395955, '中'),
#         (0.009984570768472667, '在'),
#         (0.009852627950874188, '一')]

[(0.035732269174118744, '的'),
 (0.012927703414087723, '國'),
 (0.010620050461395955, '中'),
 (0.009984570768472667, '在'),
 (0.009852627950874188, '一')]

In [13]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

## 使用Ngram來建立第一版選字系統

In [14]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        # YOUR CODE HERE
        if len(prefix) == 0:
            return self.unigram.predict_proba(prefix, top_k=top_k)
        elif len(prefix) == 1:
            return self.bigram.predict_proba(prefix, top_k=top_k)
        else:
            return self.trigram.predict_proba(prefix, top_k=top_k)
        # END YOUR CODE

In [15]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [16]:
probs = model.predict_proba('我思', top_k=10)
probs

[(0.75, '故'), (0.25, '維')]

## Demo

In [17]:
!pip install -U pip
!pip install -q ipywidgets

In [18]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')